# Setting Up Azure Monitor Integration
Import required libraries and configure Azure Monitor integration with AI Foundry project using environment variables and connection strings.

In [ ]:
!pip install -r requirements.txt

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.ai.projects.telemetry.agents import AIAgentsInstrumentor
from opentelemetry import trace

# Load environment variables
load_dotenv(override=True)

# Initialize Azure AI client with DefaultAzureCredential
credential = DefaultAzureCredential()
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

# Configure tracing with Azure Monitor
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights not enabled - enable it in your AI Foundry project's 'Tracing' tab")
    exit()
configure_azure_monitor(connection_string=application_insights_connection_string)

# Configure tracing
instrumentor = AIAgentsInstrumentor()
instrumentor.instrument(enable_content_recording=True)

# Create tracer
tracer = trace.get_tracer(__name__)

# Configuring the OpenTelemetry Tracer
Set up OpenTelemetry tracer and instrumentor for Azure AI agent telemetry with proper configuration and initialization.

In [ ]:
# Configuring the OpenTelemetry Tracer

# Import required libraries
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.ai.projects.telemetry.agents import AIAgentsInstrumentor

# Configure tracing with Azure Monitor
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights not enabled - enable it in your AI Foundry project's 'Tracing' tab")
    exit()
configure_azure_monitor(connection_string=application_insights_connection_string)

# Configure tracing
instrumentor = AIAgentsInstrumentor()
instrumentor.instrument(enable_content_recording=True)

# Create tracer
tracer = trace.get_tracer(__name__)

# Creating a KYC Agent with Tracing
Implement the setup_agent function that creates an AI agent with tracing capabilities, including span attributes and tool configuration.

In [ ]:
# Creating a KYC Agent with Tracing

# Import required libraries
from azure.ai.projects.models import FunctionTool, ToolSet
from kyc_functions import get_kyc_data, update_kyc_data

def setup_agent(project_client):
    """
    Setup the KYC agent with the required tools and instructions.
    """
    AGENT_NAME = "kyc-agent-eval"
    
    # Add span attributes for monitoring
    span = trace.get_current_span()
    span.set_attribute("agent_type", "kyc_agent")
    span.set_attribute("model", os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4"))
    
    # Build toolset
    toolset = ToolSet()
    toolset.add(FunctionTool({get_kyc_data, update_kyc_data}))
    
    # Create agent
    agent = project_client.agents.create_agent(
        model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4"),
        name=AGENT_NAME,
        instructions="""You are a helpful KYC agent. For every question, you should:
1. Retrieve KYC data from Cosmos DB using get_kyc_data
2. Provide a clear and concise answer based on the available data""",
        toolset=toolset
    )
    return agent

# Setup the agent
agent = setup_agent(project_client)

# Implementing Traced KYC Functions
Create the KYC data retrieval and update functions with proper tracing spans and attributes for monitoring.

In [ ]:
# Implementing Traced KYC Functions

# Import required libraries
import os
import json
from typing import Any, Dict
from dotenv import load_dotenv
from azure.cosmos import CosmosClient
from opentelemetry import trace

# Load environment variables
load_dotenv(override=True)

# Create tracer
tracer = trace.get_tracer(__name__)

def _get_container():
    endpoint = os.environ.get("COSMOS_ENDPOINT")
    key = os.environ.get("COSMOS_KEY")
    db_name = os.environ.get("COSMOS_DB_NAME")
    container_name = os.environ.get("COSMOS_CONTAINER_NAME")

    if not all([endpoint, key, db_name, container_name]):
        raise ValueError("Missing required Cosmos DB environment variables.")

    client = CosmosClient(endpoint, credential=key)
    db = client.get_database_client(db_name)
    container = db.get_container_client(container_name)
    return container

def get_kyc_data(person_name: str) -> str:
    with tracer.start_as_current_span("get_kyc_data") as span:
        span.set_attribute("person_name", person_name)
        try:
            print(f"Searching for KYC record matching '{person_name}'...")
            container = _get_container()
            
            # Split name into parts and create a more flexible search
            name_parts = person_name.lower().split()
            conditions = []
            params = []
            
            for i, part in enumerate(name_parts):
                param_name = f"@name{i}"
                conditions.append(f"CONTAINS(LOWER(c.full_name), {param_name})")
                params.append({"name": param_name, "value": part})
            
            query = f"SELECT * FROM c WHERE {' OR '.join(conditions)}"
            results = list(container.query_items(
                query=query, 
                parameters=params,
                enable_cross_partition_query=True
            ))
            
            if not results:
                return json.dumps({"error": f"No KYC records found matching '{person_name}'."})
            
            # Sort results by relevance (number of matching parts)
            results.sort(key=lambda x: sum(
                part in x['full_name'].lower() 
                for part in name_parts
            ), reverse=True)
            
            # Add trace attributes for query performance
            span.set_attribute("results_count", len(results))
            return json.dumps(results[0], ensure_ascii=False)
        except Exception as e:
            span.set_attribute("error", str(e))
            return json.dumps({"error": f"Exception: {str(e)}"})

def update_kyc_data(person_name: str, updated_data: Dict[str, Any]) -> str:
    with tracer.start_as_current_span("update_kyc_data") as span:
        span.set_attribute("person_name", person_name)
        span.set_attribute("update_fields", list(updated_data.keys()))
        try:
            container = _get_container()
            query = "SELECT * FROM c WHERE CONTAINS(LOWER(c.full_name), LOWER(@person_name))"
            params = [{"name": "@person_name", "value": person_name}]
            results = list(container.query_items(query=query, parameters=params, enable_cross_partition_query=True))
            
            if not results:
                return json.dumps({"error": f"No KYC record found for '{person_name}' to update."})

            record = results[0]
            for key, val in updated_data.items():
                record[key] = val

            container.upsert_item(record)
            return json.dumps({
                "message": f"KYC record updated for {record['full_name']}", 
                "record": record
            }, ensure_ascii=False)
        except Exception as e:
            span.set_attribute("error", str(e))
            return json.dumps({"error": f"Exception: {str(e)}"})

# Making Traced API Calls
Implement the ask function that makes traced API calls to the agent, capturing thread and message information.

In [ ]:
# Making Traced API Calls

# Implement the ask function that makes traced API calls to the agent, capturing thread and message information.

def ask(project_client, agent, question_text):
    """
    Ask a single question to the KYC agent and return the response.
    """
    # Create a new span for the question processing
    with tracer.start_as_current_span("process_kyc_question") as span:
        span.set_attribute("question", question_text)
        
        thread = project_client.agents.create_thread()
        span.set_attribute("thread_id", thread.id)
        
        # Send question
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=question_text
        )
        
        # Process the run
        run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
        span.set_attribute("run_status", run.status)
        
        if run.status == "failed":
            span.set_attribute("error", str(run.last_error))
            print(f"Run failed: {run.last_error}")
            return None

        # Fetch the final response from messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        latest_message = next((msg for msg in messages.data if msg.role == "assistant"), None)
        
        if latest_message and latest_message.content:
            response = latest_message.content[0].text.value.strip()
            span.set_attribute("response_length", len(response))
            return response
        return None

# Example usage
question = "What are Mustafa Suleyman's current affiliations?"
response = ask(project_client, agent, question)

print(f"\nQuestion: {question}")
print(f"Response: {response}")